# **<font color="red">Agents Workflow</font>**
- Specialized agents that control the execution flow of its sub-agents.
1. `Sequential Agent`
2. `Loop Agent`
3. `Parallel Agent`


## **<font color="blue">Parallel Agents</font>**
- The `ParallelAgent` is a workflow agent that executes its sub-agents _concurrently_. This dramatically speeds up workflows where tasks can be performed independently.
- Use `ParallelAgent` when: For scenarios prioritizing speed and involving independent, resource-intensive tasks, a `ParallelAgent` facilitates efficient parallel execution. **When sub-agents operate without dependencies, their tasks can be performed cocurrently, significantly reducing overall processing time.
- As with other workflow agents, the `ParallelAgent` is not powered by an LLM, and is thus deterministic in how it executes. That being said, workflow agents are only concerned with their execution (i.e. executing sub-agents in parallel), and not their internal logic; the tools or sub-agents of a workflow agent may or may not utilize LLMs.

In [12]:
"""
Parallel Web Research Pipeline using Google ADK
"""

import os
import asyncio
from config import config

from google.adk.models import Gemini
from google.adk.agents import LlmAgent, ParallelAgent, SequentialAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types


# ============================================================
# CONFIG
# ============================================================

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

APP_NAME = "parallel_research_app"
USER_ID = "user_1"
SESSION_ID = "session_001"

MODEL = Gemini(model="gemini-2.5-flash")


# ============================================================
# RESEARCH AGENTS
# ============================================================

researcher_agent_1 = LlmAgent(
    name="RenewableEnergyResearcher",
    model=MODEL,
    instruction="""
Research the latest advancements in renewable energy sources.
Use Google Search.
Summarize in 1-2 sentences.
Output ONLY the summary.
""",
    tools=[google_search],
    output_key="renewable_energy_result",
)

researcher_agent_2 = LlmAgent(
    name="EVResearcher",
    model=MODEL,
    instruction="""
Research the latest developments in electric vehicle technology.
Use Google Search.
Summarize in 1-2 sentences.
Output ONLY the summary.
""",
    tools=[google_search],
    output_key="ev_technology_result",
)

researcher_agent_3 = LlmAgent(
    name="CarbonCaptureResearcher",
    model=MODEL,
    instruction="""
Research the current state of carbon capture methods.
Use Google Search.
Summarize in 1-2 sentences.
Output ONLY the summary.
""",
    tools=[google_search],
    output_key="carbon_capture_result",
)


parallel_research_agent = ParallelAgent(
    name="ParallelWebResearchAgent",
    sub_agents=[
        researcher_agent_1,
        researcher_agent_2,
        researcher_agent_3,
    ],
)

merger_agent = LlmAgent(
    name="SynthesisAgent",
    model=MODEL,
    instruction="""
Combine the research results below into a structured report.

Renewable:
{renewable_energy_result}

EV:
{ev_technology_result}

Carbon Capture:
{carbon_capture_result}

Output format:

## Summary of Recent Sustainable Technology Advancements

### Renewable Energy Findings
### Electric Vehicle Findings
### Carbon Capture Findings
### Overall Conclusion

Only use provided inputs.
""",
)

root_agent = SequentialAgent(
    name="ResearchAndSynthesisPipeline",
    sub_agents=[
        parallel_research_agent,
        merger_agent,
    ],
)

runner = Runner(
    agent=root_agent,
    app_name=APP_NAME,
    session_service=InMemorySessionService(),
)


# ============================================================
# EXECUTION (NOW ACTUALLY CORRECT)
# ============================================================

async def main():

    print("\n" + "=" * 80)
    print("🚀 Running Parallel Research Pipeline")
    print("=" * 80 + "\n")

    # ✅ CREATE SESSION FIRST (THIS WAS MISSING)
    await runner.session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID,
    )

    user_message = types.Content(
        role="user",
        parts=[types.Part(text="Generate sustainable technology research report.")]
    )

    final_response = ""

    async for event in runner.run_async(
        user_id=USER_ID,
        session_id=SESSION_ID,
        new_message=user_message,
    ):
        if event.content and event.content.parts:
            for part in event.content.parts:
                if hasattr(part, "text") and part.text:
                    final_response += part.text

    print("\n" + "=" * 80)
    print("✅ FINAL REPORT")
    print("=" * 80 + "\n")
    print(final_response)


# Jupyter-safe
await main()



🚀 Running Parallel Research Pipeline


✅ FINAL REPORT

Carbon capture methods are rapidly advancing, utilizing technologies like post-combustion capture, pre-combustion, oxy-fuel combustion, and direct air capture (DAC) with improved materials such as metal-organic frameworks (MOFs) and advanced absorbents. While significant progress has been made in efficiency and cost reduction, challenges remain in scaling up these diverse approaches to meet ambitious global decarbonization targets.Recent advancements in renewable energy include high-efficiency perovskite and bifacial solar cells, larger and AI-optimized wind turbines with improved storage, enhanced geothermal systems accessing deeper heat, and modernized hydropower with advanced turbines and pumped storage. Additionally, biomass energy is evolving with advanced conversion technologies and carbon capture, while tidal and wave energy technologies are progressing with new turbine designs and projects.Latest developments in sustainabl